In [1]:
from edsl import Survey
s = Survey.pull('1646e0e9-b4a5-43c2-ab3e-3d5c1d3931ec')

In [2]:
from edsl.embeddings.embeddings_engine import EmbeddingsEngine

In [3]:
#from edsl.key_management import KeyLookupBuilder
#keys = KeyLookupBuilder().build()
#openai_key = keys['openai'].api_token

In [4]:
e = EmbeddingsEngine.from_survey(s)

In [5]:
e.tsne()

In [6]:
e.distance_matrix()

In [7]:
res = e.cluster_dbscan(target_num_clusters=5, min_samples=4)
res.summary() 

{'num_clusters': 3,
 'cluster_sizes': {0: 6, 1: 4, 2: 4},
 'num_noise': 30,
 'params': {'eps': 0.4,
  'min_samples': 4,
  'metric': 'cosine',
  'target_num_clusters': 5}}

In [8]:
view = e.view_clusters(target_num_clusters=7, width=800, height=600, label_size=13)
view

In [9]:
from edsl import QuestionFreeText, Scenario, QuestionList
scenario = Scenario({'question_texts': "".join([q.question_text for q in s.questions])})
q = QuestionFreeText(question_name = "persona", 
                     question_text = """
                     Please generate a persona for person who could answer every question in: {{ scenario.question_texts }}
                     Do not just repeat the question but instead have them describe themselves and their views.
                     """)


q_compress = QuestionFreeText(
    question_name = "compress",
    question_text = """Please compress this persona into just 5 sentences.
    Try to keep it as information-rich and dense as possible. 
    Each sentence should reveal new information.
    <original>
    {{ persona.answer }}
    </original>
    """)

q_jeopardy = QuestionList(question_name = "jeopardy", 
                          question_text = """Come up with good, intersting questions that this person could answer: 
                          {{ compress.answer }}"""
                         )
                             
results = q.add_question(q_compress).add_question(q_jeopardy).by(scenario).run()

In [10]:
new_q = results.select('answer.jeopardy').expand('answer.jeopardy').to_list()

In [11]:
new_e = EmbeddingsEngine().add_documents(new_q)

In [12]:
new_e

ID,Content,Embedding
0,"How do you assess the current state of U.S.-EU relations, particularly with G...",✓ 1536
1,What are the potential implications of increased military assistance to Ukrai...,✓ 1536
2,How do you perceive China's dual role as an opportunity and a challenge in gl...,✓ 1536
3,What strategies should the U.S. adopt to address the military threats posed b...,✓ 1536
4,"In your view, how can international cooperation be enhanced to tackle challen...",✓ 1536
5,How does your passion for travel influence your perspective on international ...,✓ 1536
6,What role do you see for political independence in shaping effective foreign ...,✓ 1536
7,How should the international community approach the issue of defending sovere...,✓ 1536
8,What lessons can be learned from the complex relationship between the U.S. an...,✓ 1536
9,How important is the role of diverse perspectives in formulating global polic...,✓ 1536


In [13]:
e.compare(new_e).cross_distance_heatmap()